# Low-Rank MLP Growth Loop Debugger

Interactive notebook for probing repeated compress→double→train cycles on a fixed training window using the shared CLI helpers.


## Notebook Outline
1. Import the shared low-rank MLP helpers from `rank_mlp_double.py` plus common utilities.
2. Configure a single training window and optimizer hyper-parameters.
3. Load the chain once, prepare the fixed dataset slice, and wire helper functions.
4. Run a loop of compress→double→train trials, logging losses/logπ errors and ranks in a pandas table each round.


In [1]:
from __future__ import annotations

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from pathlib import Path
from copy import deepcopy
from types import SimpleNamespace

from rank_mlp_double import (
    LowRankMLP,
    collect_training_range,
    prepare_training_arrays,
    logpi_l1_error,
    run_training_cycle,
)

from test_mlp import load_data


In [2]:
DATA_PATH = "data1.h5"
SIGMA_PRIOR = 1.0
SIGMA_LIK = 0.3

TRAIN_START_STEP = 0
TRAIN_STEPS = 5000
MASTER_VAL_START = 50000
MASTER_VAL_LENGTH = None  # When None use the remainder of the chain.

USE_STANDARDIZATION = False
WARM_START = True
NUM_GROWTH_TRIES = 100
GROWTH_COMPRESSION_RATIO = 0.25
RESULTS_CSV = None  # Optional CSV path if you want to persist the debug table.

LOW_RANK_ARCH = {
    "hidden_dim": 1024,
    "num_hidden_layers": 3,
    "ranks": [64, 64],
    "activation": nn.Tanh(),
    "noise_std": 0.3,
    "apply_final_activation": True,
}

TRAINING_CFG = {
    "max_adam_epochs": 1000,
    "adam_lr": 1e-3,
    "adam_patience": 100,
    "tol": 1e-5,
    "max_lbfgs_iter": 50,
    "loss_name": "l1",
    "train_loops": 40,
    "batch_size": 64,
    "loss_domain": "obs",
    "batch_growth": 1.2,
    "verbose": 1,
    "loop_improvement_pct": 0.1,
}

TRAINING_CFG_NS = SimpleNamespace(**TRAINING_CFG)

SEED = 123
np.random.seed(SEED)
torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


device(type='cuda')

In [3]:
par, obs, y_obs, chain, props, logpi_true = load_data(DATA_PATH, SIGMA_PRIOR, SIGMA_LIK)
input_dim = par.shape[1]
output_dim = obs.shape[1]

train_indices = collect_training_range(chain, props, TRAIN_START_STEP, TRAIN_STEPS)
X_train_raw, X_train_proc, y_train, logpi_train, x_mean, x_std = prepare_training_arrays(
    par, obs, logpi_true, train_indices, USE_STANDARDIZATION
)

if MASTER_VAL_LENGTH is None:
    master_val_length = None
else:
    master_val_length = min(MASTER_VAL_LENGTH, max(0, chain.shape[0] - MASTER_VAL_START))

print(f"Device: {DEVICE}")
print(f"Input dim: {input_dim}, output dim: {output_dim}")
print(f"Total train samples in window: {train_indices.size}")
print(f"Train steps: {TRAIN_STEPS}")
print(f"Master validation start: {MASTER_VAL_START}, length: {master_val_length}")


[data] Loaded 'logpi' from file.
[data] par shape   : (28324, 30)
[data] obs shape   : (28324, 52)
[data] y_obs shape : (52,)
[data] chain shape : (56646,)
[data] props shape : (56646,)
Device: cuda
Input dim: 30, output dim: 52
Total train samples in window: 2501
Train steps: 5000
Master validation start: 50000, length: None


In [4]:
def make_low_rank_model() -> LowRankMLP:
    return LowRankMLP(
        input_dim=input_dim,
        hidden_dim=LOW_RANK_ARCH["hidden_dim"],
        output_dim=output_dim,
        num_hidden_layers=LOW_RANK_ARCH["num_hidden_layers"],
        ranks=LOW_RANK_ARCH["ranks"],
        activation=LOW_RANK_ARCH.get("activation", nn.Tanh()),
        noise_std=LOW_RANK_ARCH.get("noise_std", 0.01),
        apply_final_activation=LOW_RANK_ARCH.get("apply_final_activation", True),
    )

def run_cycle(candidate_model: LowRankMLP) -> float:
    return run_training_cycle(
        candidate_model,
        X_train_proc,
        y_train,
        X_train_raw,
        logpi_train,
        DEVICE,
        TRAINING_CFG_NS,
        SIGMA_PRIOR,
        SIGMA_LIK,
        y_obs,
    )

def eval_master(candidate_model: LowRankMLP) -> float:
    return logpi_l1_error(
        candidate_model,
        par,
        obs,
        logpi_true,
        y_obs,
        chain,
        props,
        MASTER_VAL_START,
        master_val_length,
        USE_STANDARDIZATION,
        x_mean,
        x_std,
        SIGMA_PRIOR,
        SIGMA_LIK,
        DEVICE,
    )


In [5]:
records: list[dict] = []

model = make_low_rank_model()
model.to(DEVICE)

print("Running baseline training before the compression/doubling loop...")
prev_train_loss = run_cycle(model)
prev_master_error = eval_master(model)
print(f"  baseline train loss: {prev_train_loss:.4e}")
print(f"  baseline master logpi L1: {prev_master_error:.4e}")

for try_idx in range(1, NUM_GROWTH_TRIES + 1):
    print(f"\n=== Growth try {try_idx}/{NUM_GROWTH_TRIES} ===")
    start_ranks = tuple(int(r) for r in model.ranks)
    loss_from = float(prev_train_loss)
    master_from = float(prev_master_error)

    # Compress first, then expand all ranks by a factor of two.
    model.contract_ranks_by_ratio(GROWTH_COMPRESSION_RATIO)
    compressed_ranks = tuple(int(r) for r in model.ranks)

    target_ranks = LOW_RANK_ARCH["ranks"]  # [min(model.hidden_dim, max(1, r * 2)) for r in model.ranks]
    model.contract_ranks_by_amount(target_ranks)
    end_ranks = tuple(int(r) for r in model.ranks)

    train_loss = run_cycle(model)
    master_error = eval_master(model)

    record = {
        "iteration": try_idx,
        "train_loss_from": loss_from,
        "train_loss_to": float(train_loss),
        "master_logpi_from": master_from,
        "master_logpi_to": float(master_error),
        "start_ranks": start_ranks,
        "compressed_ranks": compressed_ranks,
        "end_ranks": end_ranks,
    }
    records.append(record)

    prev_train_loss = train_loss
    prev_master_error = master_error

    results_df = pd.DataFrame(records)
    display(results_df)

    if RESULTS_CSV:
        out_path = Path(RESULTS_CSV)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        results_df.to_csv(out_path, index=False)
        print(f"Saved table to {out_path}")

Running baseline training before the compression/doubling loop...
[train][loop 1/40] loss=8.042509e-03 next_lr=1.000e-03 stop=completed
[train][loop 2/40] loss=6.700181e-03 next_lr=2.500e-04 stop=adam_plateau
[train][loop 3/40] loss=3.516936e-03 next_lr=6.250e-05 stop=adam_plateau
[train][loop 4/40] loss=2.691244e-03 next_lr=6.250e-05 stop=completed
[train][loop 5/40] loss=2.640627e-03 next_lr=1.563e-05 stop=adam_plateau
[train][loop 6/40] loss=2.450036e-03 next_lr=1.563e-05 stop=completed
[train][loop 7/40] loss=2.440640e-03 next_lr=3.906e-06 stop=adam_plateau
[train][loop 8/40] loss=2.390511e-03 next_lr=3.906e-06 stop=completed
[train][loop 9/40] loss=2.390511e-03 next_lr=9.766e-07 stop=adam_lr_min
  baseline train loss: 2.3905e-03
  baseline master logpi L1: 3.6012e+00

=== Growth try 1/100 ===
[train][loop 1/40] loss=2.589250e-02 next_lr=2.500e-04 stop=adam_plateau
[train][loop 2/40] loss=3.891151e-03 next_lr=6.250e-05 stop=adam_plateau
[train][loop 3/40] loss=2.025179e-03 next_lr=

,iteration,train_loss_from,train_loss_to,master_logpi_from,master_logpi_to,start_ranks,compressed_ranks,end_ranks
0,1,0.002391,0.001103,3.601187,3.799068,"(64, 64)","(19, 17)","(64, 64)"



=== Growth try 2/100 ===
[train][loop 1/40] loss=2.519303e-02 next_lr=2.500e-04 stop=adam_plateau
[train][loop 2/40] loss=3.858073e-03 next_lr=6.250e-05 stop=adam_plateau
[train][loop 3/40] loss=2.120058e-03 next_lr=1.563e-05 stop=adam_plateau
[train][loop 4/40] loss=1.484648e-03 next_lr=3.906e-06 stop=adam_plateau
[train][loop 5/40] loss=1.322978e-03 next_lr=3.906e-06 stop=completed
[train][loop 6/40] loss=1.322978e-03 next_lr=9.766e-07 stop=adam_lr_min


,iteration,train_loss_from,train_loss_to,master_logpi_from,master_logpi_to,start_ranks,compressed_ranks,end_ranks
0,1,0.002391,0.001103,3.601187,3.799068,"(64, 64)","(19, 17)","(64, 64)"
1,2,0.001103,0.001323,3.799068,3.806651,"(64, 64)","(33, 47)","(64, 64)"



=== Growth try 3/100 ===
[train][loop 1/40] loss=2.585933e-02 next_lr=2.500e-04 stop=adam_plateau
[train][loop 2/40] loss=4.157683e-03 next_lr=6.250e-05 stop=adam_plateau
[train][loop 3/40] loss=2.256973e-03 next_lr=1.563e-05 stop=adam_plateau
[train][loop 4/40] loss=1.549897e-03 next_lr=1.563e-05 stop=completed
[train][loop 5/40] loss=1.521757e-03 next_lr=3.906e-06 stop=adam_plateau
[train][loop 6/40] loss=1.371847e-03 next_lr=3.906e-06 stop=completed
[train][loop 7/40] loss=1.371847e-03 next_lr=9.766e-07 stop=adam_lr_min


...

,iteration,train_loss_from,train_loss_to,master_logpi_from,master_logpi_to,start_ranks,compressed_ranks,end_ranks
0,1,0.002391,0.001103,3.601187,3.799068,"(64, 64)","(19, 17)","(64, 64)"
1,2,0.001103,0.001323,3.799068,3.806651,"(64, 64)","(33, 47)","(64, 64)"
2,3,0.001323,0.001372,3.806651,3.846378,"(64, 64)","(34, 59)","(64, 64)"
3,4,0.001372,0.001616,3.846378,3.891257,"(64, 64)","(30, 60)","(64, 64)"
4,5,0.001616,0.001677,3.891257,3.846516,"(64, 64)","(29, 60)","(64, 64)"
...,...,...,...,...,...,...,...,...
93,94,0.006387,0.006482,4.195091,4.244725,"(64, 64)","(12, 30)","(64, 64)"
94,95,0.006482,0.006603,4.244725,4.194356,"(64, 64)","(12, 30)","(64, 64)"
95,96,0.006603,0.007090,4.194356,4.227686,"(64, 64)","(12, 30)","(64, 64)"
96,97,0.007090,0.006401,4.227686,4.257806,"(64, 64)","(12, 30)","(64, 64)"



=== Growth try 99/100 ===
[train][loop 1/40] loss=2.582190e-02 next_lr=2.500e-04 stop=adam_plateau
[train][loop 2/40] loss=1.138408e-02 next_lr=6.250e-05 stop=adam_plateau
[train][loop 3/40] loss=8.061241e-03 next_lr=1.563e-05 stop=adam_plateau
[train][loop 4/40] loss=7.635180e-03 next_lr=3.906e-06 stop=adam_plateau
[train][loop 5/40] loss=7.526121e-03 next_lr=9.766e-07 stop=adam_lr_min


,iteration,train_loss_from,train_loss_to,master_logpi_from,master_logpi_to,start_ranks,compressed_ranks,end_ranks
0,1,0.002391,0.001103,3.601187,3.799068,"(64, 64)","(19, 17)","(64, 64)"
1,2,0.001103,0.001323,3.799068,3.806651,"(64, 64)","(33, 47)","(64, 64)"
2,3,0.001323,0.001372,3.806651,3.846378,"(64, 64)","(34, 59)","(64, 64)"
3,4,0.001372,0.001616,3.846378,3.891257,"(64, 64)","(30, 60)","(64, 64)"
4,5,0.001616,0.001677,3.891257,3.846516,"(64, 64)","(29, 60)","(64, 64)"
...,...,...,...,...,...,...,...,...
94,95,0.006482,0.006603,4.244725,4.194356,"(64, 64)","(12, 30)","(64, 64)"
95,96,0.006603,0.007090,4.194356,4.227686,"(64, 64)","(12, 30)","(64, 64)"
96,97,0.007090,0.006401,4.227686,4.257806,"(64, 64)","(12, 30)","(64, 64)"
97,98,0.006401,0.007163,4.257806,4.430007,"(64, 64)","(12, 30)","(64, 64)"



=== Growth try 100/100 ===
[train][loop 1/40] loss=2.748678e-02 next_lr=2.500e-04 stop=adam_plateau
[train][loop 2/40] loss=9.940580e-03 next_lr=6.250e-05 stop=adam_plateau
[train][loop 3/40] loss=7.092722e-03 next_lr=1.563e-05 stop=adam_plateau
[train][loop 4/40] loss=6.637627e-03 next_lr=3.906e-06 stop=adam_plateau
[train][loop 5/40] loss=6.545335e-03 next_lr=9.766e-07 stop=adam_lr_min


,iteration,train_loss_from,train_loss_to,master_logpi_from,master_logpi_to,start_ranks,compressed_ranks,end_ranks
0,1,0.002391,0.001103,3.601187,3.799068,"(64, 64)","(19, 17)","(64, 64)"
1,2,0.001103,0.001323,3.799068,3.806651,"(64, 64)","(33, 47)","(64, 64)"
2,3,0.001323,0.001372,3.806651,3.846378,"(64, 64)","(34, 59)","(64, 64)"
3,4,0.001372,0.001616,3.846378,3.891257,"(64, 64)","(30, 60)","(64, 64)"
4,5,0.001616,0.001677,3.891257,3.846516,"(64, 64)","(29, 60)","(64, 64)"
...,...,...,...,...,...,...,...,...
95,96,0.006603,0.007090,4.194356,4.227686,"(64, 64)","(12, 30)","(64, 64)"
96,97,0.007090,0.006401,4.227686,4.257806,"(64, 64)","(12, 30)","(64, 64)"
97,98,0.006401,0.007163,4.257806,4.430007,"(64, 64)","(12, 30)","(64, 64)"
98,99,0.007163,0.007526,4.430007,4.297161,"(64, 64)","(12, 30)","(64, 64)"
